In [30]:
# import libraries

import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
from dotenv import load_dotenv

import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# connect to Redshift

load_dotenv()
dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}")

In [3]:
# Use the read_sql() function to query this database tables

query = """select table_name from information_schema.tables\
            where table_schema='bootcamp1'"""
pd.read_sql_query(text(query), engine)

,table_name
0,online_transactions
1,online_transactions_test
2,stock_description


In [4]:
# Use the read_sql() function to query this table and store the output as a variable called online_transactions

query = """SELECT * FROM bootcamp1.online_transactions"""
online_transactions=pd.read_sql_query(text(query), engine)
online_transactions

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
0,536365,84029G,6,2010-12-01 08:26:00,3.39,u1785,United Kingdom
1,536366,22633,6,2010-12-01 08:28:00,1.85,u1785,United Kingdom
2,536368,22912,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom
3,536367,22748,6,2010-12-01 08:34:00,2.10,u13047,United Kingdom
4,536367,22623,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom
...,...,...,...,...,...,...,...
541905,581586,23275,24,2011-12-09 12:49:00,1.25,u13113,United Kingdom
541906,581587,22556,12,2011-12-09 12:50:00,1.65,u1268,France
541907,581587,22726,4,2011-12-09 12:50:00,3.75,u1268,France
541908,581587,23256,4,2011-12-09 12:50:00,4.15,u1268,France


In [5]:
# Use the read_sql() function to query this table and store the output as a variable called stock_description

query = """SELECT * FROM bootcamp1.stock_description"""
stock_description=pd.read_sql_query(text(query), engine)
stock_description

,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
...,...,...
3947,DCGSSGIRL,GIRLS PARTY BAG
3948,DOT,DOTCOM POSTAGE
3949,PADS,PADS TO MATCH ALL CUSHIONS
3950,POST,POSTAGE


## Exploring Data

### For online_transactions

In [6]:
online_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       541910 non-null  object 
 1   stock_code    541910 non-null  object 
 2   quantity      541910 non-null  int64  
 3   invoice_date  541910 non-null  object 
 4   price         541910 non-null  float64
 5   customer_id   541910 non-null  object 
 6   country       541910 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 28.9+ MB


In [10]:
online_transactions.isna().sum()

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
dtype: int64

In [13]:
# fix missing data

online_transactions = online_transactions.replace('', np.nan)
online_transactions.isna().sum()

invoice              0
stock_code           0
quantity             0
invoice_date         0
price                0
customer_id     135080
country              0
dtype: int64

In [15]:
online_transactions.duplicated().sum()

5270

In [17]:
online_transactions.describe()

,quantity,price
count,541910.000000,541910.000000
mean,9.552234,4.611138
std,218.080957,96.759765
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [18]:
online_transactions[online_transactions["quantity"] < 0].sort_values(by="quantity",ascending=True).head(10)

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
510457,C581484,23843,-80995,2011-12-09 09:27:00,2.08,u16446,United Kingdom
54223,C541433,23166,-74215,2011-01-18 10:17:00,1.04,u12346,United Kingdom
220324,556690,23005,-9600,2011-06-14 10:37:00,0.00,NaN,United Kingdom
192481,556691,23005,-9600,2011-06-14 10:37:00,0.00,NaN,United Kingdom
1465,C536757,84347,-9360,2010-12-02 14:23:00,0.03,u15838,United Kingdom
251851,556687,23003,-9058,2011-06-14 10:36:00,0.00,NaN,United Kingdom
101043,546152,72140F,-5368,2011-03-09 17:25:00,0.00,NaN,United Kingdom
419602,573596,79323W,-4830,2011-10-31 15:17:00,0.00,NaN,United Kingdom
347577,566768,16045,-3667,2011-09-14 17:53:00,0.00,NaN,United Kingdom
328533,565304,16259,-3167,2011-09-02 12:18:00,0.00,NaN,United Kingdom


In [19]:
online_transactions[online_transactions["price"] < 0]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
302488,A563187,B,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom
320012,A563186,B,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom


In [20]:
online_transactions["country"].value_counts()

United Kingdom          495478
Germany                   9495
France                    8558
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

### For stock_description

In [7]:
stock_description.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3952 entries, 0 to 3951
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   stock_code   3952 non-null   object
 1   description  3952 non-null   object
dtypes: object(2)
memory usage: 61.9+ KB


In [9]:
stock_description.isna().sum()

stock_code     0
description    0
dtype: int64

In [14]:
# fix missing data

stock_description = stock_description.replace('', np.nan)
stock_description.isna().sum()

stock_code     0
description    0
dtype: int64

In [16]:
stock_description.duplicated().sum()

0

In [22]:
stock_description.describe()

,stock_code,description
count,3952,3952
unique,3905,3785
top,22600,?
freq,2,47


In [21]:
stock_description.description.value_counts()

?                                     47
METAL SIGN,CUPCAKE SINGLE HOOK         6
CINAMMON SET OF 9 T-LIGHTS             2
COLUMBIAN CANDLE RECTANGLE             2
3 WHITE CHOC MORRIS BOXED CANDLES      2
                                      ..
MAGNETS PACK OF 4 SWALLOWS             1
MAGNETS PACK OF 4 CHILDHOOD MEMORY     1
MAGNETS PACK OF 4 HOME SWEET HOME      1
MAGNETS PACK OF 4 VINTAGE COLLAGE      1
SAMPLES                                1
Name: description, Length: 3785, dtype: int64

In [23]:
# looking at the data where stock codes are missing in the stock description table

query = """select count(distinct t1.stock_code)
           from bootcamp1.online_transactions t1
           left join bootcamp1.stock_description t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
            and customer_id <> ''
           """
pd.read_sql(text(query), engine)

,count
0,22


In [25]:
query = """SELECT DISTINCT t1.stock_code
            FROM bootcamp1.online_transactions AS t1
            LEFT JOIN bootcamp1.stock_description AS t2
            ON t1.stock_code = t2.stock_code
            WHERE t2.stock_code is null
            AND customer_id <> ''
            """
pd.read_sql(text(query), engine)

,stock_code
0,22686
1,21703
2,22878
3,46000P
4,46000S
5,21705
6,21704
7,D
8,22889
9,16151A


## Cleaning Data

1. Removing all customers with no customer id
2. Joining descriptions to do the online transaction table, and replaces missing values '?' with 'Unknown'
3. Removing all invoices that have the stock code 'BANK CHARGES', 'POSTAGE', 'D', 'M' and 'CRUK'
4. Removing all invoices where quantity is less than 0

In [26]:
query = """
SELECT ot.*,
        case when sd.description = '?' or sd.description is null then 'Unknown' else sd.description end as description
FROM bootcamp1.online_transactions ot
LEFT JOIN bootcamp1.stock_description sd ON ot.stock_code = sd.stock_code
WHERE ot.customer_id <> '' 
AND ot.stock_code NOT IN ('BANK CHARGES', 'POSTAGE', 'D', 'M', 'CRUK')
AND ot.quantity > 0 """

online_transaction = pd.read_sql(text(query), engine)

### Removing duplicates

In [27]:
online_transaction_cleaned = online_transaction.drop_duplicates(keep='first')

In [28]:
online_transaction_cleaned.duplicated().sum()

0

### Fixing the invoice_date field from object to datetime

In [31]:
online_transaction_cleaned.invoice_date = pd.to_datetime(online_transaction_cleaned.invoice_date, 
                                                   format = "%Y-%m-%d")

In [32]:
online_transaction_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399867 entries, 0 to 405134
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice       399867 non-null  object        
 1   stock_code    399867 non-null  object        
 2   quantity      399867 non-null  int64         
 3   invoice_date  399867 non-null  datetime64[ns]
 4   price         399867 non-null  float64       
 5   customer_id   399867 non-null  object        
 6   country       399867 non-null  object        
 7   description   399867 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.5+ MB


In [35]:
# close the connection

engine.dispose()